In [ ]:
import keras
import pandas as pd
import tensorflow as tf
import os
import string
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [ ]:
!pip install kaggle

!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

!unzip imdb-dataset-of-50k-movie-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 89% 23.0M/25.7M [00:00<00:00, 120MB/s] 
100% 25.7M/25.7M [00:00<00:00, 116MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
df = pd.read_csv("IMDB Dataset.csv")

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
# prompt: i have this error TypeError: expected string or bytes-like object can you fix it to me ?
from bs4 import BeautifulSoup
def remove_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  cleaned_text = soup.get_text()
  return cleaned_text

df['review'] = df['review'].apply(lambda x: remove_html_tags(str(x)))
df

<ipython-input-94-5a7f293eca58>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
import string

punctuation_chars = string.punctuation

def remove_punctuation(text):
  translator = str.maketrans('', '', punctuation_chars)
  return text.translate(translator)

df['review'] = df['review'].apply(remove_punctuation)

# ou alors en utilisant regex avec sub avec re.syb([a-zA-Z", " ", review])

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [ ]:
# Lowercase
df['review'] = df['review'].str.lower()

In [ ]:
df.head()

,review,sentiment
0,one reviewer mentioned watching 1 oz episode y...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter matteis love time money visually stunni...,positive


In [ ]:
nltk.download('stopwords')
english_stopwords =set(stopwords.words('english'))
nltk.download('wordnet')
lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def process_review(review):
    review = " ".join([lemmatizer.lemmatize(word) for word in review.split(" ") if word not in english_stopwords])
    return review

In [ ]:
df['review'] = df['review'].apply(process_review)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['review'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['review'])

padded_sequences = pad_sequences(sequences, maxlen=100)

print(padded_sequences)

[[75355  3806 11236 ...   785  3678   350]
 [    0     0     0 ...  1755    17   132]
 [    0     0     0 ...    31    13   112]
 ...
 [   11  1067     6 ...  3594 20395  5392]
 [  169   146   497 ... 10909  2350   602]
 [    0     0     0 ...   899   728     1]]


In [ ]:
# Alternative : on ne garde que les avis etant inferieur a 500 caractères

# X, Y = [], []
# for review, sentiment in zip(df['review'], df['sentiment']):
    # if len(review.split(" ")) < 500:
       #  X.append(review)
        # Y.append(sentiment)

# X, Y = np.array(X), np.array(Y)

In [ ]:
df['review'].head()

0    one reviewer mentioned watching 1 oz episode y...
1    wonderful little production filming technique ...
2    thought wonderful way spend time hot summer we...
3    basically there family little boy jake think t...
4    petter matteis love time money visually stunni...
Name: review, dtype: object

In [ ]:
!python3 -m nltk.downloader punkt

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df['review']

0        one reviewer mentioned watching 1 oz episode y...
1        wonderful little production filming technique ...
2        thought wonderful way spend time hot summer we...
3        basically there family little boy jake think t...
4        petter matteis love time money visually stunni...
                               ...                        
49995    thought movie right good job wasnt creative or...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary school nu...
49998    im going disagree previous comment side maltin...
49999    one expects star trek movie high art fan expec...
Name: review, Length: 50000, dtype: object

In [ ]:
# Le Tokenizer de keras avec de bons parametrages peut éliminer la suppression de la ponctuayion ainsi que l'application du lowercase dans notre precedent preprocess.
# tf.keras.preprocessing.text.Tokenizer(
    # num_words=None,
    # filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    # lower=True,
    # split=' ',
    # char_level=False,
    # oov_token=None,)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])
df.head()

,review,sentiment
0,one reviewer mentioned watching 1 oz episode y...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically there family little boy jake think t...,0
4,petter matteis love time money visually stunni...,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['sentiment'], test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (40000, 100)
y_train shape: (40000,)
X_test shape: (10000, 100)
y_test shape: (10000,)


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Learning Rate Decay
lr_decay = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_index) + 1, 128, input_length=100),
    tf.keras.layers.SpatialDropout1D(0.4),
    tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping, lr_decay])


Epoch 1/10
313/313 [==============================] - 144s 448ms/step - loss: 0.3721 - accuracy: 0.8342 - val_loss: 0.3023 - val_accuracy: 0.8767 - lr: 0.0010
Epoch 2/10
313/313 [==============================] - 134s 429ms/step - loss: 0.1799 - accuracy: 0.9338 - val_loss: 0.2820 - val_accuracy: 0.8815 - lr: 0.0010
Epoch 3/10
313/313 [==============================] - 127s 405ms/step - loss: 0.0966 - accuracy: 0.9657 - val_loss: 0.3407 - val_accuracy: 0.8782 - lr: 0.0010
Epoch 4/10
313/313 [==============================] - 116s 370ms/step - loss: 0.0550 - accuracy: 0.9809 - val_loss: 0.4696 - val_accuracy: 0.8697 - lr: 0.0010
Epoch 5/10
313/313 [==============================] - 114s 364ms/step - loss: 0.0359 - accuracy: 0.9877 - val_loss: 0.4840 - val_accuracy: 0.8631 - lr: 0.0010
Epoch 6/10
313/313 [==============================] - 115s 367ms/step - loss: 0.0167 - accuracy: 0.9951 - val_loss: 0.5839 - val_accuracy: 0.8724 - lr: 1.0000e-04
Epoch 7/10
313/313 [======================

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from tensorflow.keras.models import Model

input_layer = Input(shape=(100,), name='input_layer')
embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=128, name='embedding_layer')(input_layer)
lstm_layer = LSTM(128, return_sequences=True, name='lstm_layer')(embedding_layer)
attention_layer = Attention()([lstm_layer, lstm_layer])
dense_layer = Dense(64, activation='relu', name='dense_layer')(attention_layer) # Nouvelle couche Dense
output_layer = Dense(1, activation='sigmoid', name='output_layer')(dense_layer) # Couche de sortie après la nouvelle couche Dense
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping, lr_decay])

Epoch 1/10
313/313 [==============================] - 33s 98ms/step - loss: 0.3912 - accuracy: 0.8222 - val_loss: 0.3020 - val_accuracy: 0.8723 - lr: 0.0010
Epoch 2/10
313/313 [==============================] - 19s 61ms/step - loss: 0.1648 - accuracy: 0.9397 - val_loss: 0.3333 - val_accuracy: 0.8688 - lr: 0.0010
Epoch 3/10
313/313 [==============================] - 12s 39ms/step - loss: 0.0691 - accuracy: 0.9760 - val_loss: 0.4245 - val_accuracy: 0.8627 - lr: 0.0010
Epoch 4/10
313/313 [==============================] - 11s 34ms/step - loss: 0.0340 - accuracy: 0.9894 - val_loss: 0.5904 - val_accuracy: 0.8566 - lr: 0.0010
Epoch 5/10
313/313 [==============================] - 9s 30ms/step - loss: 0.0118 - accuracy: 0.9967 - val_loss: 0.7752 - val_accuracy: 0.8542 - lr: 1.0000e-04
Epoch 6/10
313/313 [==============================] - 9s 29ms/step - loss: 0.0071 - accuracy: 0.9980 - val_loss: 0.9495 - val_accuracy: 0.8528 - lr: 1.0000e-04
